<a href="https://colab.research.google.com/github/mcadriaans/Data_Analysis/blob/main/Hotel_Booking_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'hotel-booking-demand:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F511638%2F944030%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240606%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240606T112645Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9a7ac8f7960a91bd11ab990176b5095a352741f8b823d1307e13621ee2b04ea1d3661bf31e30bfce0da23c1947ebd6b3ca4b6aae940d22338bcfa817600838f8823d606d0539fc25031fd8978b59141e5fbaf6b01f09cca002b4fd8839ab040b3ebe4573a40e93783d8a33fb707832b0c0e663b54c5223deffd9ea16bc0146c467f0744630d457016489fbe4aae90a35d27ca72632c5e072c91310aabf168d4cc6212dfe0e9947dcacac935b4127841a63b7f5be8da13e049382e61c51ca2f9ab52d97ac0f621b930ebcd67f7b22c63e09c8fb8903b1b4f84168d37f5ab613178cfa5ecf2cec737f2accdf3d2366f8e677d9533b9f66d53d0b1ac87a2f5655a7'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1308365 bytes downloaded
Downloaded and uncompressed: hotel-booking-demand
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hotel-booking-demand/hotel_bookings.csv


<font size=6><b>Hotel Bookings EDA</font></b>

<div style="background-color:#FA8072; color: black; padding: 10px;">
    <h1 style="text-align: center; font-size: 40px;"> Load and understand the data </h1>
</div>

## Install packages

In [3]:
!pip install pycountry

In [4]:
!pip install country_converter

## Load Dependencies

In [5]:
import pandas as pd

import pycountry
import country_converter as coco
import warnings

# Data visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import seaborn as sns
import folium
from folium.plugins import HeatMap


In [6]:
warnings.filterwarnings("ignore", category=FutureWarning)

## Load the data

In [7]:
data = pd.read_csv('/kaggle/input/hotel-booking-demand/hotel_bookings.csv')

## Understanding the data

In [8]:
data.head(2)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01


In [9]:
# Give information about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [10]:
# Generate deecriptive statistic summary of all numerical variables
data.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119386.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,103050.000000,6797.000000,119390.000000,119390.000000,119390.000000,119390.000000
mean,0.370416,104.011416,2016.156554,27.165173,15.798241,0.927599,2.500302,1.856403,0.103890,0.007949,0.031912,0.087118,0.137097,0.221124,86.693382,189.266735,2.321149,101.831122,0.062518,0.571363
std,0.482918,106.863097,0.707476,13.605138,8.780829,0.998613,1.908286,0.579261,0.398561,0.097436,0.175767,0.844336,1.497437,0.652306,110.774548,131.655015,17.594721,50.535790,0.245291,0.792798
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,18.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,62.000000,0.000000,69.290000,0.000000,0.000000
50%,0.000000,69.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,179.000000,0.000000,94.575000,0.000000,0.000000
75%,1.000000,160.000000,2017.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,229.000000,270.000000,0.000000,126.000000,0.000000,1.000000
max,1.000000,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,72.000000,21.000000,535.000000,543.000000,391.000000,5400.000000,8.000000,5.000000


In [11]:
# Show all the unique hotel types in the ‘hotel’ column
data['hotel'].unique()

array(['Resort Hotel', 'City Hotel'], dtype=object)

In [12]:
# Count the unique values in the hotel column
data['hotel'].value_counts()

hotel
City Hotel      79330
Resort Hotel    40060
Name: count, dtype: int64

<div style="background-color:#FA8072; color: black; padding: 10px;">
    <h1 style="text-align: center; font-size: 40px;"> Data cleaning</h1>
</div>

In [13]:
clean_df = data.copy()
len(clean_df)

119390

## 1. Duplicates

In [14]:
# Count the number of duplicate rows
clean_df.duplicated().sum()

31994

In [15]:
# Drop duplicate rows across all columns
clean_df.drop_duplicates(inplace=True)

## 2. Missing Values

In [16]:
# Check if there are any null values
clean_df.isnull().values.any()

True

In [17]:
# Check percentage of missing values
perc_missing_vals = round((clean_df.isnull().sum() * 100) / len(clean_df) , 3)
perc_missing_vals = perc_missing_vals[perc_missing_vals > 0].sort_values(ascending=False)
perc_missing_vals

company     93.983
agent       13.951
country      0.517
children     0.005
dtype: float64

In [18]:
# Handle missing values
for col, perc in perc_missing_vals.items() :
    if perc >= 85 :
        clean_df = clean_df.drop(col, axis=1)  # Drop column
    elif perc < 0.01:
        clean_df = clean_df.dropna(subset=col)   # Drop rows
    elif clean_df[col].dtype == 'object':
        clean_df[col] = clean_df[col].fillna(clean_df[col].mode()[0])
    else:
        clean_df[col] = clean_df[col].fillna(clean_df[col].median())

clean_df.isnull().any().any()

False

## 3. Optimize Data

### 3.1. In the meal column we can interpret 'Undefined' as equivalent to 'SC'.

In [19]:
# Get the unique values in the meal column
clean_df['meal'].unique()

array(['BB', 'FB', 'HB', 'SC', 'Undefined'], dtype=object)

In [20]:
# Replace value Undefined with SC in the meal column
clean_df['meal'] = clean_df['meal'].replace('Undefined', 'SC')

In [21]:
# Count the occurrence of each unique value in the meal column
clean_df['meal'].value_counts()

meal
BB    67974
SC     9973
HB     9085
FB      360
Name: count, dtype: int64

### 3.2.Hotel reservations with zero guests (including adults, children, and babies) represent an illogical scenario. These entries likely indicate errors in data entry and can be safely removed from the dataset.

In [22]:
# Drop rows in df where 'adults', 'children' amd 'babies' columns are all 0
clean_df = clean_df[~(clean_df[['adults', 'children', 'babies']] == 0).all(axis=1)]
len(clean_df)

87226

### 3.3 Check if all the codes of the `country` column are of length 3 according to the ISO 3166-3 format and if there is missing values update to 'Unknown'.

In [23]:
# Returns unique country codes that are not of length
clean_df['country'][clean_df['country'].str.len() != 3].unique()

array(['CN'], dtype=object)

In [24]:
# Replace all occurrences of 'CN' with 'CHN' in the country column
clean_df['country'].replace('CN', 'CHN', inplace=True)

# Replace all occurrences of 0 with 'Unknown' in the country column
clean_df.loc[clean_df['country'] == 0, 'country'] = 'Unknown'

### 3.4. The `adr` column represents the Average daily rate of a room. If we want to compute it as a per person rate we can add a `guest_rate` column.

In [25]:
# Create a new column
clean_df['guest_rate'] = clean_df['adr']/(clean_df['adults'] + clean_df['children'])

### 3.5. Average daily rate (ADR) in hotel bookings should typically not be negative. It is used to measure the average revenue earned (price) for each occupied room per night. Filter that the dataFrame only includes positive values for the column `adr`

In [26]:
# Count the number of records that have a negative adr
len(clean_df[clean_df['adr'] < 0])

1

In [27]:
# Include only the rows from data where 'adr' not negative
clean_df = clean_df[clean_df['adr'] >= 0]

In [28]:
# Cleaned data set
df = clean_df.copy()
len(df)

87225

In [29]:
# Copy cleaned data to csv for Tableau visulaizations
#df.to_csv('cleaned_hotel_eda.csv', index=False)

<div style="background-color:#FA8072; color: black; padding: 10px;">
    <h1 style="text-align: center; font-size: 40px;"> Explore the data</h1>
</div>

<div style="background-color:#FFDAB9; color: black; padding: 1px;">
    <h1 style="text-align: center; font-size: 30px;"> Customer Behaviour Analysis</h1>
</div>

### <font size=5><u>Cancelled vs Non-cancelled Bookings Proportions</u></font>

In [30]:
df_bookings = df.groupby(['is_canceled']).size().reset_index()
df_bookings.columns=['is_canceled', 'count']

# Map the current values to the desired labels
df_bookings['is_canceled'] = df_bookings['is_canceled'].map({0: 'Non-cancelled', 1: 'Cancelled'})
df_bookings['%_of_total_bookings']= round(df_bookings['count'] / df_bookings['count'].sum(), 2)
df_bookings

,is_canceled,count,%_of_total_bookings
0,Non-cancelled,63220,0.72
1,Cancelled,24005,0.28


In [31]:
df_bookings['count'].sum()

87225

In [32]:
# Create bar chart showing the Canceled vs Non-canceled bookings
fig = px.bar(df_bookings, x='is_canceled', y='count', color='is_canceled',
             color_discrete_sequence=['green', 'red'],
             hover_data=['%_of_total_bookings'])

fig.update_layout(
    autosize=False,
    title=dict(
        text= 'Cancelled vs Non-cancelled bookings',
        x = 0.5,
        y = 0.93
    ),
    legend=dict(
        title='Booking Status'
    ),
    xaxis_title='',
    yaxis_title='Nr. of Bookings'

)
fig.show()

In [33]:
# Define Dataframe for Cancelled bookings: Select all the canceled bookings and copy it to df1
df1 = df[df['is_canceled']== 1].copy()

In [34]:
# Define Dataframe for Non-cancelled bookings: Select all the non-canceled bookings and copy it to df
df2 = df[df['is_canceled']== 0].copy()

###  <font size=5><u>Customer Preferences: Month With Most Bookings</u></font>

In [35]:
# Rename month column names to their abbreviation
df2['arrival_date_month'] = df2['arrival_date_month'].replace({
    'January' : 'Jan',
    'February': 'Feb',
    'March': 'Mar',
    'April': 'Apr',
    'June': 'Jun',
    'July': 'Jul',
    'August': 'Aug',
    'September': 'Sep',
    'October': 'Oct',
    'November': 'Nov',
    'December': 'Dec'
})



In [36]:
## Create a dataframe that shows monthly bookings

mnths= ['Jan', 'Feb', 'Mar', 'Apr','May', 'Jun',
        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Count the frequency of each unique month orders by actual month order
df_months = df2['arrival_date_month'].value_counts().reindex(mnths).reset_index().copy()
df_months.columns = ['month', 'bookings']

# DataFrame that contains the total bookings per month
df_months.head(2)

,month,bookings
0,Jan,3648
1,Feb,4671


In [37]:
## Create a Lollipot chart showing Booking trends per month

x = df_months['month']
y = df_months['bookings']

# Create a scatter plot for the lollipops
scatter = go.Scatter(
    x=x,
    y=y,
    mode='markers',
    marker=dict(size=20),
    name='Total Bookings',
    showlegend=False)

## Create a line plot for the sticks
lines = []  # initializes an empty list to store all the line plots
for i, month in enumerate(x):
    line = go.Scatter(
        x=[month, month],
        y=[0, y[i]],
        mode='lines',
        marker=dict(color='blue'),
        showlegend=False)

    lines.append(line)

# Combine everything together
data = [scatter] + lines
fig = go.Figure(data=data)

# Modify the hovertemplate of each data trace
for data in fig.data:
    data.hovertemplate = '%{x}: %{y}'

# Modify layout
fig.update_layout(
    title=dict(
        text=' Total Bookings: A Monthly Overview',
        x = 0.45,
        xanchor = 'center',
        y = 0.93),
   xaxis_title = 'Month',
   yaxis_title = 'Bookings')

# Show the figure
fig.show()

<font size= 5, color=darkcyan><b><u>Observations:</u></b></font>

<font size= 4>The majority of reservations occur in the months of July and August, while the fewest bookings are made at the start and close of the year.</font>


### <font size=5><u>Monthly Breakdown of Bookings by Hotel Type</u></font>

In [38]:
# Count the frequency for each combination of month and hotel
df_hotels_month = df2.groupby(['arrival_date_month', 'hotel']).size().reset_index()
# Rename columns
df_hotels_month.columns=['month', 'hotel', 'bookings']

# Convert 'month' column into a categorical type with a specific order
df_hotels_month['month'] = pd.Categorical(df_hotels_month['month'], categories=mnths, ordered=True)
# Sort by month
df_hotels_month.sort_values(by='month', inplace=True)
df_hotels_month.head(6)

,month,hotel,bookings
8,Jan,City Hotel,1961
9,Jan,Resort Hotel,1687
6,Feb,City Hotel,2618
7,Feb,Resort Hotel,2053
15,Mar,Resort Hotel,2190
14,Mar,City Hotel,3469


In [39]:
# Create a line plot of monthly booking trends for each hotel type
fig = px.line(df_hotels_month, x='month', y='bookings', color='hotel',
              color_discrete_sequence=['crimson', 'blue'])

# Update legend names
newnames={'City Hotel' : 'City', 'Resort Hotel':'Resort'}

# Define hovetext for each trace
hovertext={'City Hotel': '%{x}<br>Nr. of Bookings: %{y}',
           'Resort Hotel': '%{x}<br>Nr. of Bookings: %{y}'}

# Update hovertext for each trace
fig.for_each_trace(lambda t: t.update(name=newnames[t.name],
                                      legendgroup=newnames[t.name],
                                      hovertemplate=hovertext[t.name]))

# Modify layout
fig.update_layout(
    title= dict(
    text=' Monthly Booking Trends by Hotel',
    x=0.45,
    xanchor='center',
    y=0.93
),
legend=dict(
    title='Hotel Type'
),
xaxis_title = 'Month',
yaxis_title = 'Bookings'
)

# Display the plot
fig.show()

<font size= 5 color=darkcyan><b><u>Observations:</u></b></font>


* <font size= 4>City Hotels have more monthly bookings and overall bookings than Resort Hotels.</font>
* <font size= 4>Both City and Resort hotels booking trends are somewhat similar, peaking and falling around the same time.</font>


### <font size=5><u>Customer Preferences: Length of Stay</u></font>

In [40]:
# For each booking in the dataset find the total nr of nights
df_stays = df2[['hotel','stays_in_week_nights', 'stays_in_weekend_nights']].copy()
df_stays.loc[:,'total_nights'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']

# Find the number of not cancelled bookings for each combination of hotel type and total nights
df_nights = df_stays.groupby(['hotel', 'total_nights']).size().reset_index()
df_nights.columns = ['hotel', 'total_nights', 'count']

# Create 2 seperate dataFrames for city hotel type and resort hotel type
df_cty_stay = df_nights[df_nights['hotel'] == 'City Hotel']
df_rt_stay = df_nights[df_nights['hotel'] == 'Resort Hotel']


In [41]:
# Create a Bar chart that shows the count of stays for each total number of nights at the City Hotel
trace1=go.Bar(
    x= df_cty_stay['total_nights'],
    y=df_cty_stay['count'],
    name= 'City Hotel',
    marker=dict(color='crimson')
)
# Create a Bar chart that shows the count of stays for each total number of nights at the Resort Hotel
trace2=go.Bar(
    x= df_rt_stay['total_nights'],
    y=df_rt_stay['count'],
    name= 'Resort Hotel',
    marker=dict(color='blue')
)
# Combine traces to be displayed on the figure
data_stay = [trace1,trace2]
fig = go.Figure(data=data_stay)
# Update legend names
newnames = {'Resort Hotel': 'Resort', 'City Hotel':'City'}

# Define hovertext for each trace
hovertext={'Resort Hotel':'Total nights: %{x}<br>Nr. of Guests: %{y}</br>',
           'City Hotel' : 'Total nights: %{x}<br>Nr. of Guests: %{y}</br>'}

# Update hovertext for each trace
fig.for_each_trace(lambda t: t.update(name=newnames[t.name],
                                      legendgroup=newnames[t.name],
                                      hovertemplate=hovertext[t.name]))
fig.update_layout(
    autosize=False,
    title=dict(
        text='Number of Guests by Hotel and Duration of Stay',
        x = 0.45,
        xanchor = 'center',
        y = 0.93
    ),
    legend=dict(
        title='Hotel Type'
    ),
    xaxis_title = 'Total Nights',
    yaxis_title = ' Number of Guests'
)
fig.show(rendered='colab')

<font size= 5 color=darkcyan><b><u>Observations:</u></b></font>

* <font size= 4>At City Hotels, most guests chose to stay for 3 nights, whereas at Resort hotels, most guests preferred a 1 night stay.Possible reasons why many guests would only spend one night at a Resort Hotel:
    
> <font size= 4>Many local guests might choose to indulge in the distinctive experience offered by a Resort Hotel for a single night. An instance of this could be newlyweds who, after celebrating their wedding at the resort, decide to spend their first night as a married couple there.

> <font size= 4>The resort hotel could simply be a brief stopover for guests who have multiple destinations in their travel itinerary. The resort hotels could be closer to popular tourist attractions.

<font size= 4>Contrastingly, City Hotels often see guests booking longer stays for several reasons:

<font size= 4>Their strategic location within the city provides easy access to various parts of the city, making them an ideal base for guests keen on city exploration.
<font size= 4>City Hotels are also a popular choice among business travelers who need to attend multi-day events such as conferences or meetings.

<div style="background-color:#FFDAB9; color: black; padding: 1px;">
    <h1 style="text-align: center; font-size: 30px;"> Geographical Analysis</h1>
</div>

Analysis will consider only non-cancelled bookings.

In [42]:
# Create a deep copy of df2 (non-cancelled bookings), df will remain unaffected by changes in guests_df
guests_df = df2.copy()

### <font size=5><u>Distribution of Guests By Country</u></font>

In [43]:
# Count guests that did not cancel their bookings for each country
guests_by_country = guests_df ['country'].value_counts().reset_index()
# Rename the columns of the 'guests_by_country' dataFrame
guests_by_country.columns = ['country', 'count']
total_guests = guests_by_country['count'].sum()
# Calculate the percentage of guests from each country
guests_by_country['guests_in_%'] = round((guests_by_country['count'] / total_guests * 100),2)
# Calculate the cumulative sum of the percentage of guests from each country
guests_by_country['guests_cum_%']= guests_by_country['guests_in_%'].cumsum()
guests_by_country.head()

,country,count,guests_in_%,guests_cum_%
0,PRT,17991,28.46,28.46
1,GBR,8439,13.35,41.81
2,FRA,7091,11.22,53.03
3,ESP,5382,8.51,61.54
4,DEU,4332,6.85,68.39


In [44]:
# Create a Pareto chart to visualize teh distribution of guests by country
fig = go.Figure()
fig.add_trace(go.Bar(x= guests_by_country['country'], y = guests_by_country['count'], name='Guests', marker={'color':'darkviolet'}))
fig.add_trace(go.Scatter(x= guests_by_country['country'], y = guests_by_country['guests_cum_%'],
                         mode = 'lines+markers', name='Cumlative % Guests',  marker={'color':'green'}))

fig.update_layout(
    title=dict(
        text="Pareto Chart: Distribution of Guests by Country",
        x=0.5,   # x=0 is left, x=0.5 is center, x=1 is right
        y=0.85,  # y=0 is bottom, y=0.5 is middle, y=1 is top
        xanchor='center',  # sets the x position anchor
        yanchor='top'      # sets the y position anchor
     )
)

# Display the figure using the 'colab' renderer
fig.show()

<font size= 5 color=darkcyan><b><u>Observations:</u></b></font>

* <font size=4>The number of guests from Portugal is just below 18k and is significantly higher than the other countries(ranges between 1 and 8440.)</font>
* <font size=4>Portugal, Great Britain and France account for 50% of the guests.</font>

### <font size=5><u>Map Displaying the Proportion of Guests from each Country</u></font>

In [45]:
# Specify initial center coordinates and zoom level
basemap = folium.Map(location=[0, 0], zoom_start=2, prefer_canvas=True)

# Create a thematic map where areas are colored in relation to the number of bookings
guests_map = px.choropleth(
    guests_by_country,
    locations='country',
    color='count',
    hover_name='country',
    hover_data=['guests_in_%'],  # specify necessary columns
)
# Set the title and centre it
guests_map.update_layout(
    title = dict(
        text='Guests Proportion and Count By Country',
        x=0.5,             # x=0 is left, x=0.5 is center, x=1 is right
        y=0.93,            # y=0 is bottom, y=0.5 is middle, y=1 is top
        xanchor='center',  # sets the x position anchor
        yanchor='top',     # sets the y position anchor
    )
)
guests_map.show()

<font size= 5 color=darkcyan><b><u>Observations:</u></b></font>

<font size= 4>This map provides a visual representation of global booking patterns, allowing for easy comparison between different regions.</font>


<font size= 4>The color gradient represents the number of confirmed bookings, with darker shades indicating more bookings and lighter shades indicating fewer bookings.</font>

* <font size= 4>Most of Africa has less than 1000 confirmed bookings.</font>

* <font size= 4>Western Europe have placed the most bookings with majority of guests orginating from  Portugual.</font>



### <font size=5><u>Proportion of Guests by Continent</u></font>

In [48]:

# Take a alpha-3 country code as an argument and return the corresponding country name
def get_country_name(code):
    try:
        return pycountry.countries.get(alpha_3=code).name
    except:
        return 'Unknown'

# Create a new column with the country names for each country code
guests_by_country['country_names'] = guests_by_country['country'].apply(get_country_name)

# Take a country name as an argument and return the corresponding continent
def get_continent_name(country):
    if country != 'Unknown':
        return coco.convert(names=country, to='continent')
    else:
        return 'Unknown'

# Create a new column with the continent for each country name
guests_by_country['continent'] = guests_by_country['country_names'].apply(get_continent_name)

guests_by_country.head()

,country,count,guests_in_%,guests_cum_%,country_names,continent
0,PRT,17991,28.46,28.46,Portugal,Europe
1,GBR,8439,13.35,41.81,United Kingdom,Europe
2,FRA,7091,11.22,53.03,France,Europe
3,ESP,5382,8.51,61.54,Spain,Europe
4,DEU,4332,6.85,68.39,Germany,Europe


In [49]:
# Calculate the total count and percentage of guests for each continent
continents_grouped  = guests_by_country.groupby(['continent'])[['count', 'guests_in_%']].sum().reset_index()
continents_grouped.head()

,continent,count,guests_in_%
0,Africa,573,0.86
1,America,3146,4.94
2,Antarctica,2,0.00
3,Asia,2402,3.77
4,Europe,56745,89.73


In [50]:
# Create a pie chart that shows the distribution of guests across  continents
fig = px.pie(continents_grouped, values='guests_in_%', names='continent')

# Update layout
fig.update_layout(
    autosize = False,
    title = dict(
        text = 'Proportion of Bookings Across Continents',
        x = 0.5, # position the x
        y = 0.97, # position the y
        xanchor = 'center', # sets the x position anchor
        yanchor = 'top' # sets the y position anchor
    )
)

# Display chart
fig.show()

<font size=5 color=darkcyan><b><u>Observations:</u></b></font>

<font size=4>This chart provides a visual representation of where guests are coming from, with a clear majority coming from Europe. The “Unknown” category suggests that for some guests, the continent of origin could not be determined.</font>
* <font size=4>The majority of guests, 89.2%, are from Europe, the majority of the revenue
  comes from western Europe.</font>
* <font size=4>Other continents represent smaller percentages: America (4.95%), Asia (3.78%), Africa (0.861%), and Oceania (0.661%).
* <font size=4>A small percentage (0.541%) of the guest distribution is unknown.</font>

<div style="background-color:#FFDAB9; color: black; padding: 1px;">
    <h1 style="text-align: center; font-size: 30px;"> Price Analysis</h1>
</div>

### <font size=5><u> Monthly Guest Rate by Hotel Type</u></font>

In [51]:
# Create a df that contains only successful(not canceled) bookings at the 'Resort Hotel'
df2_resort = df2[df2['hotel'] == 'Resort Hotel']

# Create a df that contains only successful(not canceled) bookings at the 'City Hotel'
df2_city = df2[df2['hotel'] == 'City Hotel']

# Create a new dataFrame that contains the avg 'adr' for each month for resort hotels
resort_adr = df2_resort.groupby(['arrival_date_month'])['guest_rate'].mean().reset_index()
resort_adr.columns = ['month','resort_guest_rate'] # rename columns

# Create a new dataFrame that contains the avg 'adr' for each month for resort hotels
city_adr = df2_city.groupby(['arrival_date_month'])['guest_rate'].mean().reset_index()
city_adr.columns = ['month',  'city_guest_rate'] # rename columns

# Merges two dataFrames based on a common column 'month'
# Create a dataframe that shws the month and avg prices for resort and city
df_hotels_adr =resort_adr.merge(city_adr, on='month', how='outer')
df_hotels_adr

,month,resort_guest_rate,city_guest_rate
0,Apr,43.793905,59.466500
1,Aug,83.278460,58.294493
2,Dec,35.388722,48.348214
3,Feb,30.633775,49.489087
4,Jan,30.246077,50.243977
5,Jul,70.405494,57.399957
6,Jun,56.717048,66.807196
7,Mar,34.070546,52.075199
8,May,42.511496,72.258780
9,Nov,30.021267,56.673946


In [52]:
#mnths= ['Jan', 'Feb', 'Mar', 'Apr','May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Convert 'month' column into a categorical type with a specific order defined by 'mnths'
df_hotels_adr['month'] = pd.Categorical(df_hotels_adr['month'], categories=mnths, ordered=True)
# Sort by month
df_hotels_adr.sort_values(by='month', inplace=True)

# Add a month_num column that represents months as integers
df_hotels_adr['month_num'] = range(1, 13)

df_hotels_adr.head(6)

,month,resort_guest_rate,city_guest_rate,month_num
4,Jan,30.246077,50.243977,1
3,Feb,30.633775,49.489087,2
7,Mar,34.070546,52.075199,3
0,Apr,43.793905,59.466500,4
8,May,42.511496,72.258780,5
6,Jun,56.717048,66.807196,6


In [ ]:
# Create a line plot that shows how 'resort_guest_rate' and 'city_guest_rate' changes over the months


fig = px.line(df_hotels_adr, x = 'month_num', y = ['resort_guest_rate', 'city_guest_rate'], color_discrete_sequence=['blue', 'crimson'])

# Update legend names
newnames = {'resort_guest_rate' : 'Resort', 'city_guest_rate' : 'City'}

# Define new hover text for each trace
hovertext = {'resort_guest_rate' : 'Month: %{customdata} <br>Price: €%{y:.2f}</br>',
             'city_guest_rate' :   'Month: %{customdata} <br>Price: €%{y:.2f}</br>'}

# Update hovertext for each trace
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = hovertext[t.name],
                                      customdata = df_hotels_adr['month'])
                  )
# Update layout of the figure
fig.update_layout(
    autosize=False,
    title=dict(
        text ='Monthly Guest Rate by Hotel Type',
        x = 0.5, # set x position
        y = 0.93,# set y position
        xanchor='center'# horizontal alignment wrt x
    ),
    legend=dict(
            title='Hotel Type'
    )
)
# Specify the tick values on the xaxes
fig.update_xaxes(
    tickvals=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
)

# Update axis labels
fig.update_xaxes(
    title='Month'
)
fig.update_yaxes(
    title='Price [Euro]'
)

fig.show()



<font size=5 color=darkcyan><b><u>Observations:</u></b></font>

<font size=4>This graph compares the price trends of resort and city hotels throughout the year.</font>

* <font size=4>Resort hotels guest rates per night ranges from 30 to just under 84 dollars per person.</font>
* <font size=4>City hotels guest rates per night ranges from 48 to just under 73 dollars per person.</font>
* <font size=4>The resort prices are highest around the 8th month(August, end of Summer), while the city hotel prices peak twice a year the 5th month(May, end of Spring) and the 9th month(September, beginning of Autumn).</font>

### <font size=5><u>Guest Rate Distribution by Room and Hotel Type</u></font>

In [53]:
# Select rows and columns from df where bookings were not canceled and the three columns
df_rooms=df2[['assigned_room_type', 'guest_rate', 'hotel']]
# Sort by room type
df_rooms.sort_values(by='assigned_room_type', ascending=True)
df_rooms.head(2)

,assigned_room_type,guest_rate,hotel
0,C,0.0,Resort Hotel
1,C,0.0,Resort Hotel


In [54]:
# Create a boxplot to show the distribution of ‘guest_rate’ for each ‘assigned_room_type’ by hotel type
fig = px.box(df_rooms, x='assigned_room_type', y= 'guest_rate', color='hotel',
             color_discrete_sequence=['crimson', 'blue'])

# Define new trace names
newnames = {'Resort Hotel' : 'Resort', 'City Hotel' : 'City'}

# Define new hovertext for each trace
hovertext = {'Resort Hotel' : 'Room type: %{x}<br>Price: €%{y:.2f}</br>',
             'City Hotel' : 'Room type: %{x}<br>Price: €%{y:.2f}</br>'}

# Apply a function to each trace in the figure to update name,legendgroup and hovertemplate attributes
fig.for_each_trace( lambda t: t.update(name=newnames[t.name],
                                       legendgroup = newnames[t.name],
                                       hovertemplate=hovertext[t.name]))

# Modify various aspects of the layout
fig.update_layout(
    title=dict(
        text='Guest Rate Distribution by Room and Hotel Type',
        x = 0.5,
        y= 0.93
    ),
    legend=dict(
        title='Hotel Type'
    ),
    xaxis_title='Room Type',
    yaxis_title = 'Price [Euro]'

)
fig.show()

<font size=5 color=darkcyan><b><u>Observations:</u></b></font>

<font size=4>There is a noticeable variation in the guest rate across different room types which is to be expected as the guest rate generally is influenced by a variety of factors such as room size, amenities, view, demand and more.</font>

<font size=4>For both city and resort hotels most of the guest rates are positively skewed, so the median can provide a more accurate representaton of the 'typical' guest rate.</font>

* <font size=4>The room types A, C, D, and E tend to have higher costs and exhibit greater fluctuations in price at City Hotels.</font>
* <font size=4>Room types B, F, and G tend to be more costly and exhibit larger price variations at Resort Hotels.</font>

### <font size=5><u> Total Revenue by Room and Hotel Type</u></font>

Recall:
```
df2 : non_cancelled bookings
df2_resort : non_cancelled bookings for Resort Hotels
df2_city: non_cancelled bookings for City Hotels

```

In [55]:
# Unique 'assigned_room_type' for non-cancelled bookings
df2['assigned_room_type'].unique()

array(['C', 'A', 'D', 'E', 'G', 'F', 'I', 'B', 'H', 'K'], dtype=object)

In [56]:
# Find the total 'adr' for each 'assigned_room_type' for Resort hotel type
resort_room_adr = df2_resort.groupby('assigned_room_type')['adr'].sum().reset_index()
# Rename columns
resort_room_adr.columns = ['room_type', 'resort_adr']
# Find the total 'adr' for each 'assigned_room_type' for City hotel type
city_room_adr = df2_city.groupby('assigned_room_type')['adr'].sum().reset_index()
# Rename columns
city_room_adr.columns = ['room_type', 'city_adr']

# Create df_rooms_adr that includes all 'room_type' values from both resort_room_adr and city_room_adr
df_rooms_adr = resort_room_adr.merge(city_room_adr, on = 'room_type', how = 'outer')
df_rooms_adr.head(2)

,room_type,resort_adr,city_adr
0,A,760008.29,2249065.85
1,B,15343.25,119882.30


In [57]:
# The Bar plot shows show the ‘resort_adr’ and ‘city_adr’ for each ‘room_type'
fig = px.bar(df_rooms_adr, x='room_type', y=['resort_adr', 'city_adr'], color_discrete_sequence=['blue', 'crimson'])

# Update legend names
newnames = {'resort_adr': 'Resort', 'city_adr':'City'}

# Define hovertext for each trace
hovertext={ 'resort_adr': 'Room: %{x} <br>Total Revenue: €%{y:,.2f}</br>',
            'city_adr': 'Room: %{x} <br>Total Revenue: €%{y:,.2f}</br>'}

# Update hovertext for each trace
fig.for_each_trace(lambda t: t.update(name=newnames[t.name],
                                      legendgroup=newnames[t.name],
                                      hovertemplate=hovertext[t.name])
                   )

fig.update_layout(
    autosize = False,
    title=dict(
        text='Total Revenue by Room and Hotel Type',
        x = 0.45,
        xanchor = 'center',
        y = 0.93
    ),
    legend=dict(
        title='Hotel Type'
    ),
    xaxis_title = 'Room Type',
    yaxis_title = 'Total Revenue'
)

fig.show()

<font size=5 color=darkcyan><b><u>Observations:</u></b></font>

<font size=4>In general, city hotels appear to generate higher revenues compared to resort hotels across all room types. For both resort and city hotels it appears that room type A are the most profitable.</font>

### <font size=5><u> Total Revenue by Market Segment</u></font>

<font size=4>The dataset includes a `market_segment`column. Analyzing revenue by market segment could provide insights into which segments are most profitable.</font>

In [58]:
# Show all the unique values for market segment
df2['market_segment'].unique()

array(['Direct', 'Corporate', 'Online TA', 'Offline TA/TO',
       'Complementary', 'Groups', 'Aviation'], dtype=object)

In [59]:
# Total Average Daily Rate 'adr' for each market segment
df2_market = df2.groupby('market_segment')['adr'].sum().reset_index()


In [60]:
# Create a pie chart to show 'adr' across the different market segments
fig = px.pie(df2_market, values='adr', names='market_segment', hole=.5,
             color_discrete_sequence=['indigo','purple','magenta','orchid','plum','violet','lavender'])

for data in fig.data:
    data.hovertemplate = '%{label}<br>Total Revenue: <br>€%{value:,}'

# Modify layout
fig.update_layout(
    autosize=False,
    title=dict(
        text=' Proportion of Total Revenue by Market Segment',
        x=0.5,
        y=0.93),
)
fig.show(rendered='colab')

<font size=5 color=darkcyan><b><u>Observations:</u></b></font>

<font size=4>The donut chart provides a visual representation of how total revenue is distributed across different market segments, with “Online TA” being the most significant contributor.</font>

* <font size=4>Online TA: This segment brings in the largest proportion revenue, exceeding 3 million Euros.</font>
* <font size=4>The remaining segments contribute smaller portions to the total revenue and have a combined revenue just over 500k.</font>

<font size=4>Most of the revenue comes from bookings made through a travel agency (online and offline)</font>

### <font size=5><u> Impact of Cancellations on Revenue</u></font>

In [61]:
# Show the total revenue lost each year due to cancelations
df_rev = df1.groupby(['arrival_date_year', 'hotel'])['adr'].sum().reset_index()

In [62]:
# Create a grouped bar chart
fig = px.bar(df_rev, x='adr', y='arrival_date_year',color='hotel',
             barmode='group', orientation='h', color_discrete_sequence=['crimson', 'blue'])

# Update legend names
newnames = {'Resort Hotel': 'Resort', 'City Hotel':'City'}

# Define hovertext for each trace
hovertext={'Resort Hotel':'%{y}<br>Revenue Lost: €%{x:,.2f}',
            'City Hotel':'%{y}<br>Revenue Lost: €%{x:,.2f}'}

# Update hovertext for each trace
fig.for_each_trace(lambda t: t.update(name=newnames[t.name],
                                      legendgroup=newnames[t.name],
                                      hovertemplate=hovertext[t.name]))
fig.update_layout(
    autosize = False,
    title=dict(
        text='Annual Loss in Revenue from Cancellations',
        x = 0.45,
        xanchor = 'center',
        y = 0.93
    ),
    legend=dict(
        title='Hotel Type'
    ),
    xaxis_title = 'Revenue Lost [Euros]',
    yaxis_title = 'Year'
)

# Display the figure in Google colab
fig.show(rendered='colab')

<font size=5 color=darkcyan><b><u>Observations:</u></b></font>

<font size=4>The chart represents the annual loss in revenue from cancellations for two types of hotels - City and Resort, over three years (2015, 2016, and 2017).</font>

<font size=4>There's an increase in lost revenue due to cancellations for both hotel types over the years, but it's more pronounced for City hotels.</font>

<font size=4>There can be several reasons why City hotels experience more cancellations than resort hotels:</font>

* <font size=4>City hotels typically serve business travelers who may frequently alter their plans, resulting in a higher number of cancellations. Conversely, resort hotels are usually frequented by holidaymakers whose plans tend to be more fixed.</font>

* <font size=4>In this dataset, City hotels recorded a total of 53274 reservations, while Resort hotels had 33955 reservations. The likelihood of cancellations increases with the number of bookings.</font>